In [81]:
import networkx as nx
import pandas as pd
import random
import torch
import pickle
from sklearn.model_selection import train_test_split
import numpy as np

In [82]:
import torch
from torch_geometric.utils.convert import from_networkx
from pygod.detector import DOMINANT, OCGNN, CONAD, GAE, AnomalyDAE
from pygod.metric import eval_average_precision, eval_roc_auc, eval_f1, eval_precision_at_k, eval_recall_at_k
import pickle
import time

In [83]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

# Load Dataset

In [41]:
unsw_labeled_path = "C:\\Users\\asus\\Documents\\nids-pcap-dataset\\unsw_parquet_used_dataset\\unsw_labeled.parquet"

In [42]:
unsw = pd.read_parquet(unsw_labeled_path)

In [43]:
unsw.info()
unsw.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 125180 entries, 1 to 490022
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   source_ip         125180 non-null  object  
 1   destination_ip    125180 non-null  object  
 2   source_port       125180 non-null  object  
 3   destination_port  125180 non-null  object  
 4   info_message      125180 non-null  object  
 5   attack_category   15657 non-null   category
 6   is_malware        125180 non-null  int64   
 7   source_ip_info    125180 non-null  object  
 8   source_port_info  125180 non-null  object  
 9   dest_ip_info      125180 non-null  object  
 10  dest_port_info    125180 non-null  object  
 11  count_benign      125180 non-null  int64   
 12  count_malware     125180 non-null  int64   
dtypes: category(1), int64(3), object(9)
memory usage: 12.5+ MB


,source_ip,destination_ip,source_port,destination_port,info_message,attack_category,is_malware,source_ip_info,source_port_info,dest_ip_info,dest_port_info,count_benign,count_malware
index,,,,,,,,,,,,,
1,175.45.176.1,149.171.126.18,4657,80,GET /oKmwKoVbq HTTP/1.1,NaN,0,175.45.176.1 GET /oKmwKoVbq HTTP/1.1,4657 GET /oKmwKoVbq HTTP/1.1,149.171.126.18 GET /oKmwKoVbq HTTP/1.1,80 GET /oKmwKoVbq HTTP/1.1,1,0
2,175.45.176.3,149.171.126.18,32473,80,GET /level/15/exec/-/buffers/assigned/dump HTT...,NaN,1,175.45.176.3 GET /level/15/exec/-/buffers/assi...,32473 GET /level/15/exec/-/buffers/assigned/du...,149.171.126.18 GET /level/15/exec/-/buffers/as...,80 GET /level/15/exec/-/buffers/assigned/dump ...,1,7
6,175.45.176.0,149.171.126.17,49194,80,GET eLWfxXSPkc HTTP/1.1,NaN,0,175.45.176.0 GET eLWfxXSPkc HTTP/1.1,49194 GET eLWfxXSPkc HTTP/1.1,149.171.126.17 GET eLWfxXSPkc HTTP/1.1,80 GET eLWfxXSPkc HTTP/1.1,1,0


In [74]:
## delete character in source_port_info if it is out of index for dimension 0
unsw = unsw[unsw['source_port_info'].map(len) < 6]

In [75]:
unsw.info()
unsw.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   source_ip         0 non-null      object  
 1   destination_ip    0 non-null      object  
 2   source_port       0 non-null      object  
 3   destination_port  0 non-null      object  
 4   info_message      0 non-null      object  
 5   attack_category   0 non-null      category
 6   is_malware        0 non-null      int64   
 7   source_ip_info    0 non-null      object  
 8   source_port_info  0 non-null      object  
 9   dest_ip_info      0 non-null      object  
 10  dest_port_info    0 non-null      object  
 11  count_benign      0 non-null      int64   
 12  count_malware     0 non-null      int64   
dtypes: category(1), int64(3), object(9)
memory usage: 660.0+ bytes


,source_ip,destination_ip,source_port,destination_port,info_message,attack_category,is_malware,source_ip_info,source_port_info,dest_ip_info,dest_port_info,count_benign,count_malware
index,,,,,,,,,,,,,


# Function

In [44]:
def split_train_test(df, test_size=0.3, random_state=0):
    train, test = train_test_split(df, test_size=test_size)
    return train, test

In [69]:
def graph_modeling_1(df):
    graph = nx.Graph()
    node_features = []
    labels = []

    for source_port_info in df["source_port_info"].unique():
        graph.add_node(source_port_info)
        info_message = df[df["source_port_info"] == source_port_info]["info_message"].iloc[0]
        label = df[df["source_port_info"] == source_port_info]["is_malware"].iloc[0]
        node_features.append([float(len(info_message))])
        labels.append(label)
        
    for (source_ip, source_port), group in df.groupby(["source_ip", "source_port"]):
        for i in range(len(group) - 1):
            from_node = group.iloc[i]["source_port_info"]
            to_node = group.iloc[i+1]["source_port_info"]
            if graph.has_edge(from_node, to_node):
                graph[from_node][to_node]["weight"] += 1
            else:
                graph.add_edge(from_node, to_node, weight=1)
    return graph, node_features, labels

# Eksperimen 1

In [29]:
unsw['source_port'] = unsw.source_port.astype('int32')
unsw['destination_port']= unsw.destination_port.astype('int32')

NameError: name 'unsw' is not defined

In [57]:
train_df, test_df = split_train_test(unsw)

In [58]:
train_df.is_malware.value_counts()

is_malware
0    77233
1    10393
Name: count, dtype: int64

In [59]:
nD_train_df = train_df.drop_duplicates(subset=['info_message'], keep='first')
label_train = nD_train_df['is_malware'].to_numpy()
label_train = torch.tensor(label_train, dtype=torch.float)
value_counts = np.unique(label_train, return_counts=True)
print(value_counts)

(array([0., 1.], dtype=float32), array([1653, 5923], dtype=int64))


In [60]:
nD_train_df.is_malware.value_counts()

is_malware
1    5923
0    1653
Name: count, dtype: int64

In [69]:
test_df.is_malware.value_counts()

is_malware
0    33041
1      863
Name: count, dtype: int64

In [62]:
nD_test_df = test_df.drop_duplicates(subset=['info_message'], keep='first')
nD_test_df.is_malware.value_counts()
label_test = nD_test_df['is_malware'].to_numpy()
label_test = torch.tensor(label_test, dtype=torch.float)
value_counts = np.unique(label_test, return_counts=True)
print(value_counts)

(array([0., 1.], dtype=float32), array([ 821, 3060], dtype=int64))


In [67]:
train_graph, train_node_features = graph_modeling_2(train_df)

In [68]:
# number of nodes
print(train_graph.number_of_nodes())

7576


In [70]:
test_graph, test_node_features = graph_modeling_2(test_df)

### DOMINANT

In [75]:
pyG_train = from_networkx(train_graph)
pyG_train.x = train_node_features
pyG_test = from_networkx(test_graph)
pyG_test.x = test_node_features

In [78]:
dominant_model = DOMINANT(gpu=0, weight=0.02, num_layers=64, hid_dim=64, contamination=0.1, lr=0.001, verbose=3, epoch=100)

In [79]:
dominant_compile = dominant_model.fit(pyG_train, label_train)

Epoch 0000: Loss 4.8750 | AUC 0.6216 | Recall 0.7873 | Precision 0.7873 | AP 0.8504 | F1 0.7783 | Time 12.57
Epoch 0001: Loss 4.8739 | AUC 0.6213 | Recall 0.7874 | Precision 0.7874 | AP 0.8514 | F1 0.7874 | Time 12.25
Epoch 0002: Loss 4.8740 | AUC 0.6214 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.20
Epoch 0003: Loss 4.8726 | AUC 0.6213 | Recall 0.7874 | Precision 0.7874 | AP 0.8514 | F1 0.7874 | Time 12.20
Epoch 0004: Loss 4.8725 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.24
Epoch 0005: Loss 4.8718 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.26
Epoch 0006: Loss 4.8707 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.07
Epoch 0007: Loss 4.8701 | AUC 0.6214 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 11.98
Epoch 0008: Loss 4.8685 | AUC 0.6213 | Recall 0.7876 | Precision 0.7876 | AP 0.8514 | F1 0.7876 | Time 12.03
Epoch 0009: Loss 4.

In [80]:
dominant_ip_pred_res, dominant_ip_score_res, dominant_ip_prob_res, dominant_ip_conf_res = dominant_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)

Test: Loss 0.0021 | 

ValueError: Found input variables with inconsistent numbers of samples: [3881, 1582]

In [ ]:
f1_score_ip = eval_f1(label_test, dominant_ip_pred_res)
print(f1_score_ip)
precision = eval_precision_at_k(label_test, dominant_ip_score_res, k=1606)
print(precision)
recall = eval_recall_at_k(label_test, dominant_ip_score_res, k=1606)
print(recall)

In [73]:
def dominant(train_graph, train_node_features, label_train, test_graph, test_node_features, label_test):
    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features
    pyG_test = from_networkx(test_graph)
    pyG_test.x = test_node_features

    model = DOMINANT(gpu=0, weight=0.02, num_layers=64, hid_dim=64, contamination=0.1, lr=0.001, verbose=3, epoch=100)
    dominant_compile = model.fit(pyG_train)
    dominant_ip_pred_res, dominant_ip_score_res, dominant_ip_prob_res, dominant_ip_conf_res = dominant_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
    f1_score_ip = eval_f1(label_test, dominant_ip_pred_res)
    precision = eval_precision_at_k(label_test, dominant_ip_score_res, k=3881)
    recall = eval_recall_at_k(label_test, dominant_ip_score_res, k=3881)
    return f1_score_ip, precision, recall

In [74]:
f1 = []
precision = []
recall = []
for i in range(3):
    f1_score, precision_score, recall_score = dominant(train_graph, train_node_features, label_train, test_graph, test_node_features, label_test)
    f1.append(f1_score)
    precision.append(precision_score)
    recall.append(recall_score)

Epoch 0000: Loss 4.8750 |  | Time 12.97
Epoch 0001: Loss 4.8732 |  | Time 12.00
Epoch 0002: Loss 4.8745 |  | Time 12.29


KeyboardInterrupt: 

# Eksperimen 2

In [46]:
train_df, test_df = train_test_split(unsw)

In [51]:
nD_train_df = train_df.drop_duplicates(subset=['source_port_info'], keep='first')

In [52]:
nD_train_df.is_malware.value_counts()

is_malware
0    10052
1     5405
Name: count, dtype: int64

In [49]:
value_counts = train_df.is_malware.value_counts()
if value_counts[0] > 10398:
    benign = train_df[train_df['is_malware'] == 0].sample(n=10398)
    malicious = train_df[train_df['is_malware'] == 1].sample(n=5699)
    train_df = pd.concat([benign, malicious])
    # nD_train_df = train_df.copy()

In [53]:
train_df.is_malware.value_counts()

is_malware
0    10398
1     5699
Name: count, dtype: int64

In [54]:
test_df.is_malware.value_counts()

is_malware
0    27450
1     3845
Name: count, dtype: int64

In [55]:
value_counts = test_df.is_malware.value_counts()
if value_counts[0] > 3593:
    benign = test_df[test_df['is_malware'] == 0].sample(n=3584)
    malicious = test_df[test_df['is_malware'] == 1].sample(n=3564)
    test_df = pd.concat([benign, malicious])
    # nD_test_df = test_df.copy()

In [56]:
nD_test_df = test_df.drop_duplicates(subset=['source_port_info'], keep='first')
nD_test_df.is_malware.value_counts()

is_malware
0    3535
1    3420
Name: count, dtype: int64

In [70]:
train_graph, train_node_features, label_train = graph_modeling_1(train_df)

In [71]:
test_graph, test_node_features, label_test = graph_modeling_1(test_df)

In [72]:
pickle.dump(train_graph, open('model_graph_try/train_graph_ip_port.pkl', 'wb'))
pickle.dump(train_node_features, open('model_graph_try/train_node_features_ip_port.pkl', 'wb'))
pickle.dump(label_train, open('model_graph_try/label_train_ip_port.pkl', 'wb'))


In [73]:
pickle.dump(test_graph, open('model_graph_try/test_graph_ip_port.pkl', 'wb'))
pickle.dump(test_node_features, open('model_graph_try/test_node_features_ip_port.pkl', 'wb'))
pickle.dump(label_test, open('model_graph_try/label_test_ip_port.pkl', 'wb'))


In [84]:
train_graph = pickle.load(open('model_graph/train_graph_port.pkl', 'rb'))
label_train = pickle.load(open('model_graph/label_train_port.pkl', 'rb'))
train_node_features = pickle.load(open('model_graph/train_node_features_ip_port.pkl', 'rb'))

In [85]:
test_graph = pickle.load(open('model_graph/test_graph_port.pkl', 'rb'))
label_test = pickle.load(open('model_graph/label_test_port.pkl', 'rb'))
test_node_features = pickle.load(open('model_graph/test_node_features_port.pkl', 'rb'))

In [86]:
test_graph.number_of_nodes()

8604

In [87]:
print(len(list(nx.isolates(test_graph))))

6999


In [17]:
type(label_test)
type(test_node_features)

list

In [16]:
#convert label_test to numpy array
label_test_np = np.array(label_test)
label_test_torch = torch.from_numpy(label_test_np)

In [7]:
#value counts label train
label_train = torch.tensor(label_train, dtype=torch.float)
unique, counts = torch.unique(label_train, return_counts=True)
dict(zip(unique, counts))

C:\Users\asus\AppData\Local\Temp\ipykernel_12748\1615726038.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_train = torch.tensor(label_train, dtype=torch.float)


{tensor(0.): tensor(10070), tensor(1.): tensor(5381)}

In [18]:
train_graph.number_of_nodes()

15480

In [19]:
test_graph.number_of_nodes()

8604

In [20]:
# print num of isolated nodes
print(len(list(nx.isolates(train_graph))))
# print num of isolated nodes
print(len(list(nx.isolates(test_graph))))

11213
6999


In [21]:
test_graph.number_of_nodes()

8604

In [22]:
test_node_features

[[28.0],
 [15.0],
 [28.0],
 [15.0],
 [28.0],
 [15.0],
 [201.0],
 [15.0],
 [28.0],
 [15.0],
 [15.0],
 [28.0],
 [15.0],
 [201.0],
 [28.0],
 [15.0],
 [28.0],
 [15.0],
 [15.0],
 [15.0],
 [28.0],
 [28.0],
 [24.0],
 [24.0],
 [15.0],
 [28.0],
 [15.0],
 [21.0],
 [28.0],
 [28.0],
 [28.0],
 [28.0],
 [28.0],
 [28.0],
 [15.0],
 [15.0],
 [24.0],
 [28.0],
 [24.0],
 [28.0],
 [28.0],
 [28.0],
 [15.0],
 [24.0],
 [28.0],
 [15.0],
 [15.0],
 [28.0],
 [15.0],
 [28.0],
 [28.0],
 [15.0],
 [24.0],
 [28.0],
 [28.0],
 [15.0],
 [15.0],
 [24.0],
 [28.0],
 [15.0],
 [15.0],
 [15.0],
 [28.0],
 [28.0],
 [28.0],
 [28.0],
 [15.0],
 [193.0],
 [28.0],
 [28.0],
 [15.0],
 [28.0],
 [15.0],
 [15.0],
 [28.0],
 [28.0],
 [15.0],
 [15.0],
 [28.0],
 [15.0],
 [28.0],
 [24.0],
 [15.0],
 [15.0],
 [15.0],
 [28.0],
 [28.0],
 [15.0],
 [15.0],
 [28.0],
 [28.0],
 [15.0],
 [24.0],
 [15.0],
 [28.0],
 [15.0],
 [15.0],
 [28.0],
 [28.0],
 [15.0],
 [15.0],
 [15.0],
 [15.0],
 [15.0],
 [15.0],
 [28.0],
 [15.0],
 [28.0],
 [15.0],
 [15.0],
 [28.0]

In [23]:
import torchvision.models as models
import torch_geometric.nn as pyg_nn

## DOMINANT

In [26]:
def make_dominant_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features

    dominant_model = DOMINANT(gpu=0, weight=1, weight_decay=3, num_layers=16, hid_dim=16, contamination=0.37, lr=0.001, verbose=3, epoch=5) 
    # dominant_model = DOMINANT(hid_dim=64, num_layers=4, dropout=0.0, weight_decay=0.0, sigmoid_s=False, contamination=0.1, lr=0.004, epoch=100, gpu=-1, batch_size=0, num_neigh=-1, weight=0.5, verbose=3)
    dominant_compile = dominant_model.fit(pyG_train)
    return dominant_compile, pyG_test

In [27]:
def predict_dominant(label_test, dominant_compile, test_graph, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    dominant_ip_pred_res, dominant_ip_score_res = dominant_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear')
    
    precision_pygod = eval_precision_at_k(label_test, dominant_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, dominant_ip_score_res)
    f1_score = 2 * (precision_pygod * recall_pygod) / (precision_pygod + recall_pygod)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    return precision_pygod, recall_pygod, f1_score

In [28]:
dominant_model = make_dominant_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_dominant(label_test, dominant_model, test_graph, test_node_features)

RuntimeError: The size of tensor a (15473) must match the size of tensor b (15480) at non-singleton dimension 0

In [25]:
def predict_dominant(label_test, dominant_compile, test_graph, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    dominant_ip_pred_res, dominant_ip_score_res = dominant_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear')
    
    precision_pygod = eval_precision_at_k(label_test, dominant_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, dominant_ip_score_res)
    f1_score = 2 * (precision_pygod * recall_pygod) / (precision_pygod + recall_pygod)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    return precision_pygod, recall_pygod, f1_score

def make_dominant_model(train_graph, train_node_features, 
                        label_train, test_graph, test_node_features):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = torch.tensor(label_train)
    test_node_features = torch.tensor(test_node_features)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features

    dominant_model = DOMINANT(hid_dim=64, num_layers=4, dropout=0.0, weight_decay=0.0, sigmoid_s=False, contamination=0.1, lr=0.004, epoch=100, gpu=-1, batch_size=0, num_neigh=-1, weight=0.5, verbose=3)
    dominant_compile = dominant_model.fit(pyG_train)
    return dominant_compile, pyG_test

dominant_model = make_dominant_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_dominant(label_test, dominant_model, test_graph, test_node_features)

TypeError: make_dominant_model() missing 2 required positional arguments: 'test_graph' and 'test_node_features'

## OCGNN

In [16]:
def predict_ocgnn(label_test, ocgnn_compile, pyG_test, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(pyG_test)
    pyG_test.x = test_node_features

    ocgnn_ip_pred_res, ocgnn_ip_score_res = ocgnn_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, prob_method='linear')
    
    unique_values, counts = torch.unique(ocgnn_ip_pred_res, return_counts=True)
    print(unique_values, counts)

    precision_pygod = eval_precision_at_k(label_test, ocgnn_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, ocgnn_ip_score_res)
    f1_score = 2 * (precision_pygod * recall_pygod) / (precision_pygod + recall_pygod)
    f1_pygod = eval_f1(label_test, ocgnn_ip_pred_res)
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    print("F1 pygod: ", f1_pygod)

    return precision_pygod, recall_pygod, f1_score

In [17]:
def make_ocgnn_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train.cpu()
    pyG_train.x = train_node_features.cpu()
    label_train = label_train.cpu()

    # ocgnn_model = OCGNN(hid_dim=14, num_layers=32, weight_decay=1, 
    #                 contamination=0.37, lr=0.001, epoch=100, gpu=-1, 
    #                 verbose=3)
    ocgnn_model = OCGNN(hid_dim=64, num_layers=2, weight_decay=0.0, contamination=0.1, lr=0.004, epoch=5, gpu=-1, batch_size=0, num_neigh=-1, beta=0.5, warmup=2, eps=0.001, verbose=3)
    ocgnn_compile = ocgnn_model.fit(pyG_train)
    return ocgnn_compile


In [18]:
ocgnn_model = make_ocgnn_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score= predict_ocgnn(label_test, ocgnn_model, test_graph, test_node_features)

Epoch 0000: Loss 182891.4375 |  | Time 0.13
Epoch 0001: Loss 122219.3906 |  | Time 0.17
Epoch 0002: Loss 83138.7109 |  | Time 0.26
Epoch 0003: Loss 55200.0352 |  | Time 0.26
Epoch 0004: Loss 35886.1953 |  | Time 0.47
Test: Loss 4.0640 | AUC 0.2716 | Recall 0.3904 | Precision 0.3904 | AP 0.4185 | F1 0.4028 | Time 0.04
tensor([0, 1]) tensor([5966, 2638])
F1 score:  tensor(0.3904)
Precision:  tensor(0.3904)
Recall:  tensor(0.3904)
F1 pygod:  0.1781140861466822


In [ ]:
def predict_ocgnn(label_test, ocgnn_compile, pyG_test, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(pyG_test)
    pyG_test.x = test_node_features

    ocgnn_ip_pred_res, ocgnn_ip_score_res = ocgnn_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, prob_method='linear')

    precision_pygod = eval_precision_at_k(label_test, ocgnn_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, ocgnn_ip_score_res)
    f1_score = 2 * (precision_pygod * recall_pygod) / (precision_pygod + recall_pygod)
    f1_pygod = eval_f1(label_test, ocgnn_ip_pred_res)
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    print("F1 pygod: ", f1_pygod)

    return precision_pygod, recall_pygod, f1_score

def make_ocgnn_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train.cpu()
    pyG_train.x = train_node_features.cpu()
    label_train = label_train.cpu()

    ocgnn_model = OCGNN(hid_dim=64, num_layers=4, weight_decay=0.0, contamination=0.1, lr=0.004, epoch=100, gpu=-1, batch_size=0, num_neigh=-1, beta=0.5, warmup=2, eps=0.001, verbose=3)
    ocgnn_compile = ocgnn_model.fit(pyG_train)
    return ocgnn_compile

ocgnn_model = make_ocgnn_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score= predict_ocgnn(label_test, ocgnn_model, test_graph, test_node_features)


## GAE

In [38]:
def make_gae_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train.x = train_node_features
    label_train = label_train

    # gae_model = GAE(hid_dim=12, num_layers=12, weight_decay=3,
    #             contamination=0.37, lr=0.001, epoch=5, gpu=-1,
    #             verbose=3, recon_s=True, sigmoid_s=True)
    gae_model = GAE(hid_dim=12, num_layers=12, weight_decay=3,
                 contamination=0.37, lr=0.001, epoch=100, sigmoid_s=True, recon_s = True, gpu=-1, verbose=3)
    gae_compile = gae_model.fit(pyG_train, label_train)
    return gae_compile

In [39]:
def predict_gae(gae_compile, test_graph, test_node_features, label_test):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    gae_ip_pred_res, gae_ip_score_res = gae_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, prob_method='linear')
    f1_score_pygod = eval_f1(label_test, gae_ip_pred_res)
    precision = eval_precision_at_k(label_test, gae_ip_score_res)
    recall = eval_recall_at_k(label_test, gae_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 score pygod: ", f1_score_pygod)
    
    return f1_score_pygod, precision, recall, f1_score

In [40]:
gae_model = make_gae_model(train_graph, train_node_features, label_train)
f1_score_pygod, precision_score, recall_score, f1_score = predict_gae(gae_model, test_graph, test_node_features, label_test)

alo


IndexError: index 15473 is out of bounds for dimension 0 with size 15473

In [ ]:
def predict_gae(gae_compile, test_graph, test_node_features, label_test):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    gae_ip_pred_res, gae_ip_score_res = gae_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, prob_method='linear')
    f1_score_pygod = eval_f1(label_test, gae_ip_pred_res)
    precision = eval_precision_at_k(label_test, gae_ip_score_res)
    recall = eval_recall_at_k(label_test, gae_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 score pygod: ", f1_score_pygod)
    
    return f1_score_pygod, precision, recall, f1_score
    
def make_gae_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features
    label_train = label_train

    gae_model = GAE(hid_dim=64, num_layers=4, dropout=0.0, verbose = 3, weight_decay=0.0, recon_s=False, sigmoid_s=False, contamination=0.37, lr=0.004, epoch=100, gpu=-1, batch_size=0, num_neigh=-1)
    gae_compile = gae_model.fit(pyG_train, label_train)
    return gae_compile

gae_model = make_gae_model(train_graph, train_node_features, label_train)
f1_score_pygod, precision_score, recall_score, f1_score = predict_gae(gae_model, test_graph, test_node_features, label_test)

## CONAD

In [88]:
def make_conad_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features

    # conad_model = CONAD(hid_dim=10, num_layers=16, 
    #                     lr=0.001, weight_decay= 1, contamination=0.37,
    #                     epoch=5, gpu=-1,  
    #                     weight=1, dropout=0.2, verbose=3)
    conad_model = CONAD(epoch=5, verbose=3)
    conad_compile = conad_model.fit(pyG_train, label_train)

    return conad_compile

In [89]:
def predict_conad(conda_compile, test_graph, label_test, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features
    
    conad_ip_pred_res, conad_ip_score_res = conda_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear')
    f1_score_pygod = eval_f1(label_test, conad_ip_pred_res)
    precision = eval_precision_at_k(label_test, conad_ip_score_res)
    recall = eval_recall_at_k(label_test, conad_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    
    unique_values, counts = torch.unique(conad_ip_pred_res, return_counts=True)
    print(unique_values, counts)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 score pygod: ", f1_score_pygod)
    return f1_score_pygod, precision, recall

In [90]:
conad_model = make_conad_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_conad(conad_model, test_graph, label_test, test_node_features)

alo


IndexError: The shape of the mask [15473] at index 0 does not match the shape of the indexed tensor [15480, 1] at index 0

In [ ]:
def predict_conad(conda_compile, test_graph, label_test, test_node_features):
    
    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)

    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features
    
    conad_ip_pred_res, conad_ip_score_res = conda_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear')
    f1_score_pygod = eval_f1(label_test, conad_ip_pred_res)
    precision = eval_precision_at_k(label_test, conad_ip_score_res)
    recall = eval_recall_at_k(label_test, conad_ip_score_res)
    f1_score = 2*(precision*recall)/(precision+recall)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 score pygod: ", f1_score_pygod)
    return f1_score_pygod, precision, recall

def make_conad_model(train_graph, train_node_features, 
                        label_train):
    
    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)

    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features

    conad_model = CONAD(hid_dim=64, num_layers=4, dropout=0.0, weight_decay=0.0, contamination=0.1, weight=0.5, lr=0.004, epoch=100, gpu=-1, batch_size=0, verbose=3)
    conad_compile = conad_model.fit(pyG_train, label_train)

    return conad_compile

conad_model = make_conad_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_conad(conad_model, test_graph, label_test, test_node_features)

#### dump

In [12]:
conad_mode = CONAD(hid_dim=64, num_layers=4, dropout=0.0, weight_decay=0.0, contamination=0.1, lr=0.001, 
      epoch=5, gpu=-1, num_neigh=-1, weight=0.5, eta=0.5, margin=0.5, verbose=3)

In [14]:
train_node_features = torch.tensor(train_node_features)
label_train = torch.tensor(label_train)

pyG_train = from_networkx(train_graph)
pyG_train = pyG_train
pyG_train.x = train_node_features
label_train = label_train
conda_compile = conad_mode.fit(pyG_train)

Epoch 0000: Loss 455197.7500 |  | Time 4.96
Epoch 0001: Loss 378663.6875 |  | Time 6.63
Epoch 0002: Loss 312579.7812 |  | Time 5.38
Epoch 0003: Loss 256141.1094 |  | Time 5.40
Epoch 0004: Loss 208317.6719 |  | Time 5.30


In [16]:

test_node_features = torch.tensor(test_node_features)
label_test = np.array(label_test)
label_test = torch.tensor(label_test)

pyG_test = from_networkx(test_graph)
pyG_test = pyG_test
pyG_test.x = test_node_features

conad_ip_pred_res, conad_ip_score_res, conad_ip_prob_res, conad_ip_conf_res = conda_compile.predict(data=pyG_test, label = label_test,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(conad_ip_pred_res, return_counts=True)
print(unique_values, counts)

C:\Users\asus\AppData\Local\Temp\ipykernel_2904\2043115951.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_node_features = torch.tensor(test_node_features)


Test: Loss 40.8382 | AUC 0.8885 | Recall 0.7667 | Precision 0.7667 | AP 0.8784 | F1 0.7667 | Time 0.32
tensor([0, 1]) tensor([6331,  623])


#### cont

In [16]:
conad_model = make_conad_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_conad(conad_model, test_graph, label_test, test_node_features)

C:\Users\asus\AppData\Local\Temp\ipykernel_8392\344465882.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_node_features = torch.tensor(train_node_features)


Epoch 0000: Loss 374296.6562 | AUC 0.8944 | Recall 0.6895 | Precision 0.6895 | AP 0.8188 | F1 0.6895 | Time 4.68
Epoch 0001: Loss 183653.5781 | AUC 0.8945 | Recall 0.6895 | Precision 0.6895 | AP 0.8188 | F1 0.6895 | Time 4.59
Epoch 0002: Loss 86528.0391 | AUC 0.8945 | Recall 0.6895 | Precision 0.6895 | AP 0.8188 | F1 0.6895 | Time 4.58
Epoch 0003: Loss 42603.1992 | AUC 0.8944 | Recall 0.6895 | Precision 0.6895 | AP 0.8188 | F1 0.6895 | Time 4.90
Epoch 0004: Loss 26592.3750 | AUC 0.8944 | Recall 0.6895 | Precision 0.6895 | AP 0.8187 | F1 0.6895 | Time 4.50
Epoch 0005: Loss 23481.7227 | AUC 0.8944 | Recall 0.6895 | Precision 0.6895 | AP 0.8187 | F1 0.6895 | Time 4.71
Epoch 0006: Loss 24589.7988 | AUC 0.8944 | Recall 0.6895 | Precision 0.6895 | AP 0.8187 | F1 0.6895 | Time 4.56
Epoch 0007: Loss 25234.4531 | AUC 0.8944 | Recall 0.6895 | Precision 0.6895 | AP 0.8187 | F1 0.6895 | Time 4.57
Epoch 0008: Loss 23966.7910 | AUC 0.8944 | Recall 0.6895 | Precision 0.6895 | AP 0.8187 | F1 0.6895 | 

ValueError: Input contains NaN.

## AnomalyDAE

In [41]:
def make_anomalydae_model(train_graph, train_node_features, 
                        label_train):

    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)
    
    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features
    label_train = label_train

    # anomalydae_model = AnomalyDAE(hid_dim=12, emb_dim=4, 
    #                     lr=0.001, contamination=0.37,
    #                     epoch=5, gpu=0,
    #                     weight=1, verbose=3)
    anomalydae_model = AnomalyDAE(epoch=5, verbose=3)
    anomalydae_compile = anomalydae_model.fit(pyG_train)

    return anomalydae_compile

In [42]:
def predict_anomalydae(label_test, anomalydae_compile, pyG_test, test_node_features):

    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)
    
    pyG_test = from_networkx(pyG_test)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    anomalydae_ip_pred_res, anomalydae_ip_score_res = anomalydae_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear', )
    
    precision_pygod = eval_precision_at_k(label_test, anomalydae_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, anomalydae_ip_score_res)
    f1_score = 2*(precision_pygod*recall_pygod)/(precision_pygod+recall_pygod)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    return f1_score, precision_pygod, recall_pygod

In [44]:
anomalydae_model = make_anomalydae_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_anomalydae(label_test, anomalydae_model, test_graph, test_node_features)

halo


IndexError: index 15473 is out of bounds for dimension 0 with size 15473

In [17]:
def predict_anomalydae(label_test, anomalydae_compile, test_graph, test_node_features):

    test_node_features = torch.tensor(test_node_features)
    label_test = np.array(label_test)
    label_test = torch.tensor(label_test)
    
    pyG_test = from_networkx(test_graph)
    pyG_test = pyG_test
    pyG_test.x = test_node_features

    anomalydae_ip_pred_res, anomalydae_ip_score_res = anomalydae_compile.predict(data=pyG_test, label = label_test, return_pred=True, return_score=True, prob_method='linear', )
    
    precision_pygod = eval_precision_at_k(label_test, anomalydae_ip_score_res)
    recall_pygod = eval_recall_at_k(label_test, anomalydae_ip_score_res)
    f1_score = 2*(precision_pygod*recall_pygod)/(precision_pygod+recall_pygod)
    
    print("F1 score: ", f1_score)
    print("Precision: ", precision_pygod)
    print("Recall: ", recall_pygod)
    return f1_score, precision_pygod, recall_pygod

def make_anomalydae_model(train_graph, train_node_features, 
                        label_train):

    train_node_features = torch.tensor(train_node_features)
    label_train = np.array(label_train)
    label_train = torch.tensor(label_train)
    
    pyG_train = from_networkx(train_graph)
    pyG_train = pyG_train
    pyG_train.x = train_node_features
    label_train = label_train
    
    anomalydae_model = AnomalyDAE(emb_dim=64, hid_dim=64, num_layers=4, dropout=0.0, weight_decay=0.0, contamination=0.1, lr=0.004, epoch=100, gpu=0, batch_size=0, num_neigh=-1, verbose=3)
    anomalydae_compile = anomalydae_model.fit(pyG_train)

    return anomalydae_compile

anomalydae_model = make_anomalydae_model(train_graph, train_node_features, label_train)
precision_score, recall_score, f1_score = predict_anomalydae(label_test, anomalydae_model, test_graph, test_node_features)
